In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier

In [2]:
def get_data():
    data = np.loadtxt("iris.txt")
    iris_data = data[50:]
    return iris_data 

def get_bias(train_data, C12, C21): 
    N1 = 25
    N2 = len(train_data) - 25

    pi1 = N1 / (N1 + N2)
    pi2 = N2 / (N1 + N2)

    sigma = get_sigma(train_data)

    vector_u1, vector_u2 = get_vector_u1_and_u2(train_data_positive, train_data_negative)

    bias = -0.5 * (vector_u1 - vector_u2).T.dot(np.linalg.inv(sigma)).dot(vector_u1 + vector_u2).dot(np.log(C12 * pi2 / C21 * pi1))

    return(bias)

def get_sigma(train_data):
    sigma = 0
    n = len(train_data)
    vector_u = np.mean(train_data, axis=0)

    for i in range(n):
        sigma += (train_data[i] - vector_u).dot((train_data[i] - vector_u).T)

    sigma = 1 / (n - 1) * sigma
    return(sigma)

def get_w1_and_w2_b():
    weight = get_weight()
    bias = get_bias(train_data, C12, C21)
    w1 = weight[0, 0]
    w2 = weight[0, 1] 
    b = bias[0,0]

    return(w1, w2, b)

In [3]:
def get_train_and_test(feature):
    iris_data = get_data()

    positive_data = iris_data[:50, feature]
    positive_data = np.expand_dims(positive_data, axis=2)

    negative_data = iris_data[50:, feature]
    negative_data = np.expand_dims(negative_data, axis=2)

    train_data_positive = positive_data[:25]
    train_data_negative = negative_data[:25]

    test_data_positive = positive_data[25:]
    test_data_negative = negative_data[25:]

    train_data = np.concatenate((train_data_positive, train_data_negative), axis=0)
    test_data = np.concatenate((test_data_positive, test_data_negative), axis=0)

    train_label = ['positive'] * len(train_data_positive) + ['negative'] * len(train_data_negative)
    test_label = ['positive'] * len(test_data_positive) + ['negative'] * len(test_data_negative)

    return(train_data, test_data, train_label, test_label, 
            train_data_positive, train_data_negative, test_data_positive, test_data_negative)

def get_test_and_train(feature):
    iris_data = get_data()

    positive_data = iris_data[:50, feature]
    positive_data = np.expand_dims(positive_data, axis=2)

    negative_data = iris_data[50:, feature]
    negative_data = np.expand_dims(negative_data, axis=2)

    train_data_positive = positive_data[25:]
    train_data_negative = negative_data[25:]

    test_data_positive = positive_data[:25]
    test_data_negative = negative_data[:25]

    train_data = np.concatenate((train_data_positive, train_data_negative), axis=0)
    test_data = np.concatenate((test_data_positive, test_data_negative), axis=0)

    train_label = ['positive'] * len(train_data_positive) + ['negative'] * len(train_data_negative)
    test_label = ['positive'] * len(test_data_positive) + ['negative'] * len(test_data_negative)

    return(train_data, test_data, train_label, test_label, 
            train_data_positive, train_data_negative, test_data_positive, test_data_negative)

In [4]:
def get_vector_u1_and_u2(train_data_positive, train_data_negative):
    vector_u1 = np.mean(train_data_positive, axis=0)
    vector_u2 = np.mean(train_data_negative, axis=0)
    return(vector_u1, vector_u2)

def get_weight():
    sigma = get_sigma(train_data)
    vector_u1, vector_u2 = get_vector_u1_and_u2(train_data_positive, train_data_negative)
    vector_w = (vector_u1 - vector_u2).T.dot(np.linalg.inv(sigma))
    return(vector_w)

def get_slope():

    w1, w2, b = get_w1_and_w2_b()

    y1 = (-b - w1 * 0) / w2
    y2 = (-b - w1 * 1) / w2

    if y2 - y1 > 0:
        return 'positive'
    else:
        return 'negative'
        
def get_predict(test_data):
    TP, FN, FP, TN = 0, 0, 0, 0
    right_counts = 0

    w1, w2, b = get_w1_and_w2_b()

    for i in range(len(test_data)):

        x = test_data[i, 0]
        y = test_data[i, 1]

        hyperplane_height = (-b - w1 * x) / w2

        dis = hyperplane_height - y  

        predict = ''

        hyperplane_slope = get_slope()

        if hyperplane_slope == 'positive':
            if dis > 0:
                predict = 'negative'
            else:
                predict = 'positive'  
        else:
            if dis > 0:
                predict = 'positive'
            else:
                predict = 'negative'

        
        if predict == 'positive' :
            if test_label[i] == 'positive':
                TP += 1
                right_counts += 1
            else:
                FP += 1
        else:
            if test_label[i] == 'negative':
                TN += 1
                right_counts += 1
            else:
                FN += 1

    return TP, FP, TN, FN, right_counts

def get_acc_tpr_fpr(C12, C21):

    weight = get_weight()

    bias = get_bias(train_data, C12, C21)

    TP, FP, TN, FN, right_counts = get_predict(test_data)

    accuracy = (right_counts / len(test_data)) * 100
    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)

    return TPR, FPR, accuracy

In [5]:
def get_feature():
    feature = [[1, 0],[2, 0],[3, 0],[2, 1],[3, 1],[2, 3]]

    # for i in range(3):
    #     for j in range(i+1, 4):
    #          feature.append([i, j])
    return feature
    
def get_tpr_fpr_acc(feature):
    C21 = 1

    TPR_S1 = []
    FPR_S1 = []
    ACC_S1 = []

    TPR_S2 = []
    FPR_S2 = []
    ACC_S2 = []

    train_data, test_data, train_label, test_label, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_train_and_test(feature)

    for i in range(-1000, 1001, 1):
        C12 = 1.05 ** i

        TPR_1, FPR_1, accuracy_1 = get_acc_tpr_fpr(C12, C21)

        TPR_S1.append(TPR_1)
        FPR_S1.append(FPR_1)
        ACC_S1.append(accuracy_1)

    train_data, test_data, train_label, test_label, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_test_and_train(feature)

    for j in range(-1000, 1001, 1):
        C12 = 1.05 ** j

        TPR_2, FPR_2, accuracy_2 = get_acc_tpr_fpr(C12, C21)

        TPR_S2.append(TPR_2)
        FPR_S2.append(FPR_2)
        ACC_S2.append(accuracy_2)

    return (TPR_S1, FPR_S1, ACC_S1, TPR_S2, FPR_S2, ACC_S2)

In [7]:
# step6
for index in range(len(get_feature())):
    feature = get_feature()[index]
    print(feature)

    C21 = 1

    TPR_S1 = []
    FPR_S1 = []
    ACC_S1 = []

    TPR_S2 = []
    FPR_S2 = []
    ACC_S2 = []

    TPR = []
    FPR = []
    ACC = []

    train_data, test_data, train_label, test_label, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_train_and_test(feature)

    for i in range(-1000, 1001, 1):
        C12 = 1.05 ** i

        TPR_1, FPR_1, accuracy_1 = get_acc_tpr_fpr(C12, C21)

        TPR_S1.append(TPR_1)
        FPR_S1.append(FPR_1)
        ACC_S1.append(accuracy_1)

    train_data, test_data, train_label, test_label, train_data_positive, train_data_negative, test_data_positive, test_data_negative = get_test_and_train(feature)

    for i in range(-1000, 1001, 1):
        C12 = 1.05 ** i

        TPR_2, FPR_2, accuracy_2 = get_acc_tpr_fpr(C12, C21)

        TPR_S2.append(TPR_2)
        FPR_S2.append(FPR_2)
        ACC_S2.append(accuracy_2)

    for j in range(len(TPR_S1)):
        TPR.append((TPR_S1[j] + TPR_S2[j]) / 2)
        FPR.append((FPR_S1[j] + FPR_S2[j]) / 2)
        ACC.append(((ACC_S1[j] + ACC_S2[j]) / 2) * 100)

    sns.set()
    plt.xlabel('FPR') 
    plt.ylabel('TPR')
    sns.lineplot(x=FPR, y=TPR, markers=True, dashes=False, label='[feature_{}, feature_{}]'.format(feature[0]+1, feature[1]+1))


[1, 0]


OSError: iris.txt not found.